In [18]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input, concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array
pd.set_option("display.max_columns", 100)

In [19]:
metadata = pd.read_csv('metadata.csv')
metadata['isic_id'] = metadata['isic_id'].apply(lambda x: f"{x}.JPG")
image_path = 'HAM10000_images/'
metadata['isic_id'] = image_path + metadata['isic_id']
metadata.head()

,isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,diagnosis,diagnosis_confirm_type,image_type,lesion_id,melanocytic,sex
0,HAM10000_images/ISIC_0024306.JPG,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,45.0,NaN,benign,nevus,serial imaging showing no change,dermoscopic,IL_7252831,True,male
1,HAM10000_images/ISIC_0024307.JPG,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,50.0,lower extremity,benign,nevus,serial imaging showing no change,dermoscopic,IL_6125741,True,male
2,HAM10000_images/ISIC_0024308.JPG,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,55.0,NaN,benign,nevus,serial imaging showing no change,dermoscopic,IL_3692653,True,female
3,HAM10000_images/ISIC_0024309.JPG,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,40.0,NaN,benign,nevus,serial imaging showing no change,dermoscopic,IL_0959663,True,male
4,HAM10000_images/ISIC_0024310.JPG,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,60.0,anterior torso,malignant,melanoma,histopathology,dermoscopic,IL_8194852,True,male


In [20]:
metadata["diagnosis"]

0                             nevus
1                             nevus
2                             nevus
3                             nevus
4                          melanoma
                    ...            
11715    pigmented benign keratosis
11716                         nevus
11717             actinic keratosis
11718    pigmented benign keratosis
11719          basal cell carcinoma
Name: diagnosis, Length: 11720, dtype: object

In [21]:
metadata.columns

Index(['isic_id', 'attribution', 'copyright_license', 'age_approx',
       'anatom_site_general', 'benign_malignant', 'diagnosis',
       'diagnosis_confirm_type', 'image_type', 'lesion_id', 'melanocytic',
       'sex'],
      dtype='object')

In [22]:
metadata= metadata.drop(['attribution', 'copyright_license', 'diagnosis_confirm_type', 'anatom_site_general','image_type', 'lesion_id'], axis=1)

In [23]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11720 entries, 0 to 11719
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   isic_id           11720 non-null  object 
 1   age_approx        11337 non-null  float64
 2   benign_malignant  9042 non-null   object 
 3   diagnosis         11720 non-null  object 
 4   melanocytic       11720 non-null  bool   
 5   sex               11377 non-null  object 
dtypes: bool(1), float64(1), object(4)
memory usage: 469.4+ KB


In [24]:
metadata["diagnosis"].unique()

array(['nevus', 'melanoma', 'pigmented benign keratosis',
       'dermatofibroma', 'squamous cell carcinoma',
       'basal cell carcinoma', 'vascular lesion', 'actinic keratosis'],
      dtype=object)

In [25]:
from sklearn.model_selection import train_test_split

train, val= train_test_split(metadata, test_size=0.2,random_state=42)

In [26]:
train["diagnosis"].unique()

array(['basal cell carcinoma', 'pigmented benign keratosis', 'nevus',
       'melanoma', 'squamous cell carcinoma', 'dermatofibroma',
       'actinic keratosis', 'vascular lesion'], dtype=object)

In [27]:
val["diagnosis"].unique()

array(['nevus', 'melanoma', 'actinic keratosis', 'basal cell carcinoma',
       'pigmented benign keratosis', 'squamous cell carcinoma',
       'vascular lesion', 'dermatofibroma'], dtype=object)

In [28]:
datagen = ImageDataGenerator(rescale=1./255.)

In [29]:
s = (100,75)

train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=None,
    x_col="isic_id",
    y_col="diagnosis",  # change here
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="categorical",  # change here
    target_size=s,
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=val,
    directory=None,
    x_col="isic_id",
    y_col="diagnosis",  # change here
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="categorical",  # change here
    target_size=s,
)


Found 9376 validated image filenames belonging to 8 classes.
Found 2344 validated image filenames belonging to 8 classes.


In [30]:
train_generator.labels

[1,
 5,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 4,
 4,
 3,
 4,
 5,
 3,
 5,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 1,
 4,
 4,
 5,
 5,
 6,
 4,
 2,
 1,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 2,
 4,
 2,
 3,
 0,
 4,
 4,
 4,
 4,
 4,
 3,
 5,
 4,
 4,
 5,
 4,
 4,
 7,
 4,
 4,
 6,
 4,
 3,
 4,
 4,
 4,
 2,
 4,
 1,
 3,
 4,
 4,
 4,
 5,
 4,
 3,
 1,
 7,
 1,
 4,
 2,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 1,
 5,
 4,
 4,
 3,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 4,
 3,
 5,
 1,
 1,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 1,
 3,
 3,
 5,
 4,
 2,
 4,
 4,
 4,
 0,
 3,
 4,
 5,
 4,
 3,
 3,
 4,
 7,
 4,
 4,
 4,
 4,
 5,
 3,
 4,
 4,
 4,
 1,
 4,
 4,
 4,
 4,
 4,
 1,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 1,
 4,
 5,
 4,
 4,
 3,
 5,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 5,
 4,
 4,
 5,
 3,
 3,
 5,
 4,
 0,
 1,
 4,
 4,
 4,


In [31]:
Xtrain=train.drop(["isic_id", "diagnosis"], axis=1)
Xval=val.drop(["isic_id", "diagnosis"], axis=1)

In [32]:
train["diagnosis"].unique()

array(['basal cell carcinoma', 'pigmented benign keratosis', 'nevus',
       'melanoma', 'squamous cell carcinoma', 'dermatofibroma',
       'actinic keratosis', 'vascular lesion'], dtype=object)

In [33]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import backend as K

# Define a function to calculate f1 score
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 75, 3))

# Freeze all layers in the base model except the last four
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Create a new model on top of the base model
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
num_classes = len(train_generator.class_indices)  # change here
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score])

# Create a callback that saves the best model observed on the validation data
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_f1_score', verbose=1, save_best_only=True, mode='max')

# Train the model
model.fit(train_generator, validation_data=valid_generator, epochs=50, callbacks=[checkpoint])


2024-02-25 01:42:43.264039: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 01:42:43.596333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 01:42:43.596397: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 01:42:43.602362: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 01:42:43.602470: I external/local_xla/xla/stream_executor

Epoch 1/50


2024-02-25 01:42:46.804843: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-25 01:42:47.141464: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-25 01:42:47.267749: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-25 01:42:48.652248: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2cb80f1a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-25 01:42:48.652302: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-02-25 01:42:48.708621: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708805568.942419  780855 de

293/293 [==============================] - ETA: 0s - loss: 0.9835 - accuracy: 0.6719 - f1_score: 0.6658
Epoch 1: val_f1_score improved from -inf to 0.70001, saving model to best_model.h5


/home/himanshu/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


293/293 [==============================] - 31s 89ms/step - loss: 0.9835 - accuracy: 0.6719 - f1_score: 0.6658 - val_loss: 0.8673 - val_accuracy: 0.6898 - val_f1_score: 0.7000
Epoch 2/50
293/293 [==============================] - ETA: 0s - loss: 0.8095 - accuracy: 0.7186 - f1_score: 0.7168
Epoch 2: val_f1_score improved from 0.70001 to 0.70292, saving model to best_model.h5
293/293 [==============================] - 24s 82ms/step - loss: 0.8095 - accuracy: 0.7186 - f1_score: 0.7168 - val_loss: 0.9014 - val_accuracy: 0.7167 - val_f1_score: 0.7029
Epoch 3/50
 43/293 [===>..........................] - ETA: 16s - loss: 0.7091 - accuracy: 0.7544 - f1_score: 0.7489

KeyboardInterrupt: 